In [ ]:
import scanpy as sc
import seaborn as sns
import os
from scipy.stats import median_abs_deviation
import numpy as np
from matplotlib import pyplot as plt
from cellbender.remove_background.downstream import load_anndata_from_input_and_output

In [ ]:
def is_outlier(adata, metric: str, nmads=2, upper=None, lower=None):
    M = adata.obs[metric]
    if upper is not None and lower is not None:
        outlier = (M < np.median(M) - lower * median_abs_deviation(M)) | (
            np.median(M) + upper * median_abs_deviation(M) < M
        )
    else:
        outlier = (M < np.median(M) - nmads * median_abs_deviation(M)) | (
            np.median(M) + nmads * median_abs_deviation(M) < M
        )
    return outlier
#defining a function for median absolute deviation formally defined below:

## QC 

We graph various aspects of the data such as % mt, counts, and umi. Filtering out cells that are low quality or otherwise impacting analysis. Note doublet detection is done late

In [ ]:
adata = load_anndata_from_input_and_output(
    input_file='raw_feature_bc_matrix.h5',
    output_file='cellbender/cellbender.h5',
    input_layer_key='raw',  # this will be the raw data layer
)
adata=adata[adata.obs['cell_probability'] > 0.5] # Keep cells that we are at least 50% confident are real
adata.var_names_make_unique() # some genes have duplicated names for some reason, here we make unique


In [ ]:
adata.var["mt"] = adata.var_names.str.startswith("mt-") # annotates mitochondrial genes 
sc.pp.calculate_qc_metrics(
    adata, qc_vars=["mt"], inplace=True, log1p=True
) # calculates qc metrics. add the custom mt, which tells it to calculate the same metrics (counts and such) specifically for mt genes
adata

In [ ]:
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
             jitter=0.4, multi_panel=True)
#graphing number of counts, pct mt and genes
# NOTE: n_genes_by_counts translates to the number of genes with at least one count in that cell
#multi panel so each has its own y axis

In [ ]:
sc.pl.scatter(adata, "total_counts", "n_genes_by_counts", color="pct_counts_mt", )

In [ ]:
counts_lower=2
counts_upper=2
p1 = sns.displot(adata.obs["log1p_total_counts"], bins=100, kde=True)
plt.axvline(np.mean(adata.obs["log1p_total_counts"])-counts_lower*median_abs_deviation(adata.obs["log1p_total_counts"]),  color="red")
plt.axvline(np.mean(adata.obs["log1p_total_counts"])+counts_upper*median_abs_deviation(adata.obs["log1p_total_counts"]), color="red")

#Plotting cutoffs with 2 median absolute reviation cuttoffs

In [ ]:
genes_lower=2
genes_upper=2

p2 = sns.displot(adata.obs["log1p_n_genes_by_counts"], bins=100, kde=True)
plt.axvline(np.median(adata.obs["log1p_n_genes_by_counts"])-genes_lower*median_abs_deviation(adata.obs["log1p_n_genes_by_counts"]),  color="red")
plt.axvline(np.median(adata.obs["log1p_n_genes_by_counts"])+genes_upper*median_abs_deviation(adata.obs["log1p_n_genes_by_counts"]), color="red")

#Plotting cutoffs with 2 median absolute deviation cuttoffs

$\text{MAD}=\text{Median}(|X_i-\tilde{X}|)$

In [ ]:
adata.obs["outlier"] = (
    is_outlier(adata, "log1p_total_counts", upper=counts_upper, lower=counts_lower) | is_outlier(adata, "log1p_n_genes_by_counts", upper=genes_upper, lower=genes_lower))
adata.obs.outlier.value_counts()

In [ ]:
adata.obs["mt_outlier"] = ( adata.obs["pct_counts_mt"] > 5)
adata.obs.mt_outlier.value_counts()

In [ ]:
print(f"Total number of cells: {adata.n_obs}")
adata = adata[(~adata.obs.outlier) & (~adata.obs.mt_outlier)].copy()

print(f"Number of cells after filtering of low quality cells: {adata.n_obs}")

In [ ]:
p1 = sc.pl.scatter(adata, "total_counts", "n_genes_by_counts", color="pct_counts_mt")

In [ ]:
from ipylab import JupyterFrontEnd

app = JupyterFrontEnd()
app.commands.execute('docmanager:save')
#saving so html writes properly

In [ ]:
adata.write_h5ad("qc_filtered.h5ad")
os.system('jupyter nbconvert --to html QC.ipynb')